In [17]:
import pandas as pd
import numpy as np
from glob import glob


In [18]:

file_lst = glob("raw/*.csv")


In [19]:
class RES:
    def __init__(self, features=[]):
        self.ft_n = len(features)
        self.features=features
        self.RES = np.empty((1, self.ft_n))
        self.RES_len=0
        self.RES_cap=1

    def add(self,x):
        if isinstance(x, pd.DataFrame):
            x = x.to_numpy()        
        size = x.shape[0]
        if (size + self.RES_len > self.RES_cap):
            while (size + self.RES_len > self.RES_cap):
                self.RES_cap *= 2
            
            newRes = np.empty((self.RES_cap, self.ft_n))
            newRes[0:self.RES_len,:] = self.RES[0:self.RES_len,:]
            self.RES = newRes
        self.RES[self.RES_len:self.RES_len+size, :] = x
        self.RES_len += size

    def getDf(self):
        return pd.DataFrame(self.RES[0:self.RES_len,:], columns=self.features)

    def __len__(self):
        return self.RES_len
        
    def __str__(self):
        return self.RES.__str__()


In [20]:

n_normal = 0
n_attack = 0
req_ft=['duration','src_bytes','dst_bytes','missed_bytes','src_pkts','dst_pkts','label']
print(len(req_ft))
r = RES(req_ft)
for f in file_lst:    
    df = pd.read_csv(f, low_memory=False)  
    df = df.loc[:, df.columns.isin(req_ft)]
    size = df.shape[0]
    
    at_df = df[df['label'] == 1]
    nr_df = df[df['label'] == 0]
    n_nr = nr_df.shape[0]
    n_normal += n_nr
    n_attack += at_df.shape[0]    
    print(len(r))    
    
    rng = np.random.default_rng()
    r.add(rng.choice(df.to_numpy(), size=max(0,int(size*0.01)), replace=False))
    del df
    


7
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
206396


In [23]:
print (f"NORMAL: {n_normal} ATTACK: {n_attack}")

NORMAL: 755124 ATTACK: 20884485


In [31]:
df = r.getDf()
df.to_csv("processed/imbalanced_dataset.csv", index=False)

In [36]:


df = pd.read_csv("processed/imbalanced_dataset.csv")
N=df.shape[0]
train_size=int((N*3)/4)
print(f"N: {N}, TS:{train_size}")
test_df = df.loc[train_size:N].sample(frac=1)
df = df.loc[0:train_size]

N: 216396, TS:162297


In [38]:
test_df.to_csv('processed/test_data.csv', index=False)

In [39]:
from imblearn.over_sampling import ADASYN
Y = df['label'].to_numpy().astype(np.uint8)
X = df.drop('label', axis=1).to_numpy()
df[df['label'].isnull()].head()
print(Y[0:100])



[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [40]:
from imblearn.over_sampling import ADASYN
X_r, Y_r = ADASYN().fit_resample(X, Y)
res=np.empty((X_r.shape[0], df.shape[1]))
res[:, 0:-1] = X_r
res[:, -1] = Y_r
pd.DataFrame(res, columns = df.columns).to_csv("processed/balanced_dataset.csv", index=False)

In [41]:
np.bincount(Y_r)

array([158317, 157590])

In [42]:
np.random.shuffle(res)


In [43]:
pd.DataFrame(res, columns = df.columns).to_csv("processed/train_data.csv", index=False)

In [44]:
pd.DataFrame(res).tail(15)

,0,1,2,3,4,5,6
315892,0.00000,0.0,0.0,0.0,0.0,1.0,0.0
315893,0.00000,0.0,0.0,0.0,1.0,0.0,0.0
315894,0.00000,0.0,0.0,0.0,1.0,0.0,0.0
315895,0.00000,0.0,0.0,0.0,1.0,0.0,0.0
315896,0.00000,0.0,0.0,0.0,1.0,0.0,0.0
315897,63.56950,0.0,0.0,0.0,3.0,2.0,1.0
315898,0.00000,0.0,0.0,0.0,1.0,0.0,1.0
315899,0.00000,0.0,0.0,0.0,1.0,0.0,1.0
315900,0.00000,0.0,0.0,0.0,1.0,0.0,1.0
315901,0.00000,0.0,0.0,0.0,1.0,0.0,0.0
